In [1]:
import numpy as np

from Solver import Solver
from data_utils.data import get_CIFAR10_data

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape
from classifiers.cnn import ThreeLayerConvNet
num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}



X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import matplotlib.pyplot as plt

plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

NameError: name 'solver' is not defined

In [ ]:
%load_ext autoreload

%autoreload 2

from random import randrange
from classifiers import cnn_huge as cnn


num_train = 500

sample = randrange(0,(49000-1)-num_train)

small_data = {
  'X_train': data['X_train'],#[:num_train],
  'y_train': data['y_train'],#[:num_train],
  'X_val': data['X_val'],#[:num_train],
  'y_val': data['y_val'],#[:num_train],
}

small_data['X_train'] -= small_data['X_train'].mean(axis=0)
small_data['X_val'] -= small_data['X_val'].mean(axis=0)

#print small_data['X_train']
res = cnn.ResNet(weight_scale=5.4e-02,reg=0.00)
solver = Solver(res, small_data,
                    update_rule='adam',
                    optim_config={
                      'learning_rate': 1e-3,
                      'stride': 1
                    },
                    verbose=True,
                    num_epochs=10, batch_size=64,
                    print_every=1)

solver.train()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from classifiers import cnn_huge as cnn
bn_solvers = {}
solvers = {}
weight_scales = np.logspace(-4, 0, num=20)
for i, weight_scale in enumerate(weight_scales):
  print 'Running weight scale %d / %d' % (i + 1, len(weight_scales))
  bn_model = cnn.ResNet(reg=0.5, weight_scale=weight_scale, use_batchnorm=True)
  #model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, use_batchnorm=False)

  bn_solver = Solver(bn_model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-4,
                  },
                  verbose=True, print_every=1)
  bn_solver.train()
  bn_solvers[weight_scale] = bn_solver

  #solver = Solver(model, small_data,
  #                num_epochs=10, batch_size=50,
  #                update_rule='adam',
  #                optim_config={
  #                  'learning_rate': 1e-3,
  #                },
  #                verbose=False, print_every=200)
  #solver.train()
  #solvers[weight_scale] = solver

Running weight scale 1 / 20
(Iteration 1 / 100) loss: 2.302406
(Epoch 0 / 10) train acc: 0.112000; val_acc: 0.114000
(Iteration 2 / 100) loss: 2.342012
(Iteration 3 / 100) loss: 2.352511
(Iteration 4 / 100) loss: 2.318581
(Iteration 5 / 100) loss: 2.294104
(Iteration 6 / 100) loss: 2.258667
(Iteration 7 / 100) loss: 2.422350
(Iteration 8 / 100) loss: 2.320573
(Iteration 9 / 100) loss: 2.312049
(Iteration 10 / 100) loss: 2.212682
(Epoch 1 / 10) train acc: 0.178000; val_acc: 0.106000
(Iteration 11 / 100) loss: 2.276698
(Iteration 12 / 100) loss: 2.337473
(Iteration 13 / 100) loss: 2.259002
(Iteration 14 / 100) loss: 2.280259
(Iteration 15 / 100) loss: 2.187314
(Iteration 16 / 100) loss: 2.192568
(Iteration 17 / 100) loss: 2.248953
(Iteration 18 / 100) loss: 2.164723
(Iteration 19 / 100) loss: 2.234350
(Iteration 20 / 100) loss: 2.209394
(Epoch 2 / 10) train acc: 0.210000; val_acc: 0.132000
(Iteration 21 / 100) loss: 2.135870
(Iteration 22 / 100) loss: 2.193397
(Iteration 23 / 100) loss: 

In [41]:
import matplotlib.pyplot as plt
best_train_accs, bn_best_train_accs = [], []
best_val_accs, bn_best_val_accs = [], []
final_train_loss, bn_final_train_loss = [], []
i = 0
print len(bn_solvers)
for ws in weight_scales:
  i += 1
  if i < 21:
      pass
      #best_train_accs.append(max(solvers[ws].train_acc_history))
      bn_best_train_accs.append(max(bn_solvers[ws].train_acc_history))
  
      #best_val_accs.append(max(solvers[ws].val_acc_history))
      bn_best_val_accs.append(max(bn_solvers[ws].val_acc_history))
  
      #final_train_loss.append(np.mean(solvers[ws].loss_history[-100:]))
      bn_final_train_loss.append(np.mean(bn_solvers[ws].loss_history[-100:]))

print len(bn_best_train_accs)   
plt.subplot(3, 1, 1)
plt.title('Best val accuracy vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Best val accuracy')
#plt.semilogx(weight_scales, best_val_accs, '-o', label='baseline')
plt.semilogx(weight_scales[:20], bn_best_val_accs, '-o', label='batchnorm')
plt.legend(ncol=2, loc='lower right')

plt.subplot(3, 1, 2)
plt.title('Best train accuracy vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Best training accuracy')
#plt.semilogx(weight_scales, best_train_accs, '-o', label='baseline')
plt.semilogx(weight_scales[:20], bn_best_train_accs, '-o', label='batchnorm')
plt.legend()

plt.subplot(3, 1, 3)
plt.title('Final training loss vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Final training loss')
#plt.semilogx(weight_scales, final_train_loss, '-o', label='baseline')
plt.semilogx(weight_scales[:20], bn_final_train_loss, '-o', label='batchnorm')
plt.legend()

plt.gcf().set_size_inches(10, 15)
plt.show()

20
20


In [35]:
print weight_scales

[  1.00000000e-04   1.62377674e-04   2.63665090e-04   4.28133240e-04
   6.95192796e-04   1.12883789e-03   1.83298071e-03   2.97635144e-03
   4.83293024e-03   7.84759970e-03   1.27427499e-02   2.06913808e-02
   3.35981829e-02   5.45559478e-02   8.85866790e-02   1.43844989e-01
   2.33572147e-01   3.79269019e-01   6.15848211e-01   1.00000000e+00]
